# Examine Clusters

This notebook is for examining clusters generated by the second step of the analysis (phase 2). The clustering algorithm will be applied to the data and plots will be displayed showing the results. This is useful for tuning the various clustering parameters. Note that data generated here is NOT saved. This is only for testing. 

First we import all the things

In [ ]:
import sys
sys.path.append('..')
from spyral.core.config import load_config
from spyral.core.workspace import Workspace
from spyral.core.point_cloud import PointCloud
from spyral.core.clusterize import form_clusters, join_clusters_depth, cleanup_clusters
import h5py as h5
import matplotlib.pyplot as plt
import numpy.random as random
import numpy as np

Now we'll load the point clouds generated by phase one. This is very similar to the previous example where we loaded traces, so please reference that section if anything is unclear.

First load the config and the workspace

In [ ]:
config = load_config('../local_config.json')
# Tweak some parameters
# config.trace.peak_threshold = 1

# Create our workspace
ws = Workspace(config.workspace)

Now use the workspace to load the point cloud file

In [ ]:
run_number = config.run.run_min
point_file = h5.File(ws.get_point_cloud_file_path(run_number), 'r')

cloud_group: h5.Group = point_file.get('cloud')
min_event = cloud_group.attrs['min_event']
max_event = cloud_group.attrs['max_event']

Now load a random cloud from the file

In [ ]:
event = random.randint(min_event, max_event)
# event = 27216
print(f'Event {event}')
event_data = cloud_group[f'cloud_{event}']
cloud = PointCloud()
cloud.load_cloud_from_hdf5_data(event_data[:].copy(), event)
print(f'Cloud size: {len(cloud.cloud)}')

### Analysis

Now that we have our cloud were ready to cluster! The first step returns a list of clusters identified by the algorithm.

In [ ]:
clusters = form_clusters(cloud, config.cluster)

We can then plot the clusters together to check the performance of the algorithm

In [ ]:
for cluster in clusters:
    plt.scatter(cluster.point_cloud.cloud[:, 0], cluster.point_cloud.cloud[:, 1], s=3, label=f'label {cluster.label}')
plt.xlabel('x(mm)')
plt.ylabel('y(mm)')
plt.xlim(-300.0, 300.0)
plt.ylim(-300.0, 300.0)
plt.legend()
plt.show()

Above shows the x-y projection. Below are several other projections over which the clustering is performed. Typically this first stage will identify a whole bunch of cluster chunks, due to the varying density of trajectory points. So we need to join together these clusters into larger clusters

In [ ]:
for cluster in clusters:
    plt.scatter(cluster.point_cloud.cloud[:, 2], cluster.point_cloud.cloud[:, 0], s=3, label=f'label {cluster.label}')
plt.xlabel('z(mm)')
plt.ylabel('x(mm)')
plt.xlim(0.0, 1000.0)
plt.ylim(-300.0, 300.0)
plt.legend()
plt.show()

In [ ]:
for cluster in clusters:
    plt.scatter(cluster.point_cloud.cloud[:, 2], cluster.point_cloud.cloud[:, 1], s=3, label=f'label {cluster.label}')
plt.xlabel('z(mm)')
plt.ylabel('y(mm)')
plt.xlim(0.0, 1000.0)
plt.ylim(-300.0, 300.0)
plt.legend()
plt.show()

In [ ]:
for cluster in clusters:
    plt.scatter(np.linalg.norm(cluster.point_cloud.cloud[:, :3], axis=1), cluster.point_cloud.cloud[:, 4], s=3, label=f'label {cluster.label}')
plt.xlabel('R(mm)')
plt.ylabel('Integrated Charge(unitless)')
plt.legend()
plt.show()

Now we join the clusters!

In [ ]:
joined_clusters = join_clusters_depth(clusters, config.cluster)

Now we can again plot our clusters

In [ ]:
for cluster in joined_clusters:
    plt.scatter(cluster.point_cloud.cloud[:, 0], cluster.point_cloud.cloud[:, 1], s=3, label=f'label {cluster.label}')
plt.xlabel('x(mm)')
plt.ylabel('y(mm)')
plt.xlim(-300.0, 300.0)
plt.ylim(-300.0, 300.0)
plt.legend()
plt.show()

Now you should see well defined trajectory clusters! If you don't, try tweaking some of the parameters or cycling to a different point cloud. Some additional projections are shown below for the joined clusters.

In [ ]:
for cluster in joined_clusters:
    plt.scatter(cluster.point_cloud.cloud[:, 2], cluster.point_cloud.cloud[:, 0], s=3, label=f'label {cluster.label}')
plt.xlabel('z(mm)')
plt.ylabel('x(mm)')
plt.xlim(0.0, 1000.0)
plt.ylim(-300.0, 300.0)
plt.legend()
plt.show()

In [ ]:
for cluster in joined_clusters:
    plt.scatter(cluster.point_cloud.cloud[:, 2], cluster.point_cloud.cloud[:, 1], s=3, label=f'label {cluster.label}')
plt.xlabel('z(mm)')
plt.ylabel('y(mm)')
plt.xlim(0.0, 1000.0)
plt.ylim(-300.0, 300.0)
plt.legend()
plt.show()

In [ ]:
for cluster in joined_clusters:
    plt.scatter(np.linalg.norm(cluster.point_cloud.cloud[:, :3], axis=1), cluster.point_cloud.cloud[:, 4], s=3, label=f'label {cluster.label}')
plt.xlabel('R(mm)')
plt.ylabel('Integrated Charge(unitless)')
plt.legend()
plt.show()

Finally, a cleanup pass is run on the joined clusters to reduce noise and smooth the trajectory. Note that we change types here. Previously our clusters were of type LabeledCloud, a temporary holding type. Now our clusters are of type Cluster, so the semantics change a small amount.

In [ ]:
cleaned_clusters = cleanup_clusters(joined_clusters, config.cluster)

We can again plot our projections to examine the results

In [ ]:
for cluster in cleaned_clusters:
    plt.scatter(cluster.data[:, 0], cluster.data[:, 1], s=3, label=f'label {cluster.label}')
plt.xlabel('x(mm)')
plt.ylabel('y(mm)')
plt.xlim(-300.0, 300.0)
plt.ylim(-300.0, 300.0)
plt.legend()
plt.show()

In [ ]:
for cluster in cleaned_clusters:
    plt.scatter(cluster.data[:, 2], cluster.data[:, 0], s=3, label=f'label {cluster.label}')
plt.xlabel('z(mm)')
plt.ylabel('x(mm)')
plt.xlim(0.0, 1000.0)
plt.ylim(-300.0, 300.0)
plt.legend()
plt.show()

In [ ]:
for cluster in cleaned_clusters:
    plt.scatter(cluster.data[:, 2], cluster.data[:, 1], s=3, label=f'label {cluster.label}')
plt.xlabel('z(mm)')
plt.ylabel('y(mm)')
plt.xlim(0.0, 1000.0)
plt.ylim(-300.0, 300.0)
plt.legend()
plt.show()

In [ ]:
for cluster in cleaned_clusters:
    plt.scatter(np.linalg.norm(cluster.data[:, :3], axis=1), cluster.data[:, 3], s=3, label=f'label {cluster.label}')
plt.xlabel('R(mm)')
plt.ylabel('Integrated Charge(unitless)')
plt.legend()
plt.show()

### Conclusion

We've now generated clusters from our point clouds and tested the parameters, so now you can take these parameters and run the full phase 2 analysis. The next step is perfom basic physics analysis and estimate some pararameters (phase 3).